In [19]:
import numpy as np
from collections import defaultdict

In [311]:
m = [[True, True, False, True, False, True, False, True, True, False],
     [True, True, False, False, False, False, True, True, True, False],
     [True, True, False, False, False, False, False, False, False, True],
     [False, True, False, False, False, False, True, True, False, False]]

m = [[True, False, True, False, False, True, True, True],
     [True, False, True, False, False, False, True, False],
     [True, True, True, False, False, False, True, False],
     [True, False, True, False, False, False, True, False],
     [True, False, True, False, False, True, True, True]]

m = [[True, False, True],
     [False, True, False],
     [True, False, True]]

# m = [[True, True, True]]

m = [[True, False],
     [False, True]]

matrix = np.array(m, dtype=bool)
#matrix = np.random.randint(0, 2, [1, 10], bool)
prevShape = np.zeros([matrix.shape[0] + 1, matrix.shape[1] + 1], dtype=bool)

In [292]:
def generateMatrix(case: bool, on: tuple[list], baseShape: list):
    if case == True:
        patterns = np.array([[True, False, False, False],
                             [False, True, False, False],
                             [False, False, True, False],
                             [False, False, False, True]])
        
    else:
        patterns = np.array([[False, False, False, False], # 0
                             [True, True, False, False],   # 2
                             [True, False, True, False],
                             [True, False, False, True],
                             [False, True, True, False],
                             [False, True, False, True],
                             [False, False, True, True],
                             [True, True, True, False],    # 3
                             [True, True, False, True],
                             [True, False, True, True],
                             [False, True, True, True],
                             [True, True, True, True]])     # 4
    
    res = np.zeros([len(patterns), baseShape[0], baseShape[1]], dtype=bool)
    res[:, on[0], on[1]] = patterns
    
    return res                   

In [293]:
def innerJoin(A: list[list], B: list[list], on: list):
    res = np.array([np.zeros([ A[0].shape[0], A[0].shape[1] ], dtype=bool)])
    for i in range(len(A)):
        mask = np.equal(B[:, on[0], on[1]], A[i][on]).all(axis=1) 
        res = np.vstack([res, A[i] | B[mask]])
    return res[1:]

In [312]:
%%time

res = []
for i in range(matrix.shape[0]):
    for j in range(matrix.shape[1]):
        cells = ([i, i, i + 1, i + 1], [j, j + 1, j, j + 1])
        combs = generateMatrix(case = matrix[i, j], on = cells, baseShape = prevShape.shape)
        if (i, j) == (0, 0): 
            res = combs
            continue
            
        cross = ([i, i, i + 1], [j, j + 1, j])
        if i == 0:
            cross[0].pop(1)
            cross[1].pop(1)
        if j == 0:
            cross[0].pop(2)
            cross[1].pop(2)
            
        res = innerJoin(combs, res, on = cross)

        
len(res)

CPU times: total: 0 ns
Wall time: 968 µs


12

In [295]:
%%time

transitionMatrix = [[[1, 0, 1],
                     [0, 2, 1],
                     [1, 2, 1]],
                    
                    [[0, 2, 0],
                     [1, 0, 0],
                     [0, 0, 0]]]

initial = [[2, 6, 4],
           [2, 2, 0]]

state = initial[int(matrix[0, 0])]
for i in range(matrix.shape[0]):
    for j in range(1, matrix.shape[1]):
        case = int(matrix[i, j])
        nextState = [0, 0, 0]
        for k in range(len(state)):
            for m in range(len(nextState)):
                nextState[m] += transitionMatrix[case][k][m] * state[k]
        state = nextState
        
sum(state)        

CPU times: total: 0 ns
Wall time: 0 ns


464

In [270]:
%%time

cases = [{ 0: {0: 1, 2: 1}, 1: {1: 2, 2: 1}, 2: {0: 1, 1: 2, 2: 1} },
         { 0: {1: 2}, 1: {0: 1} } ]              # {right: {left: count}}

rightValuesInitial = [{0: 2, 1: 6, 2: 4}, 
                     {0: 2, 1: 2, 2: 0}]

rightValues = rightValuesInitial[int(matrix[0, 0])]


for i in range(matrix.shape[0]):
    for j in range(1, matrix.shape[1]):
        newRightValues = {0: 0, 1: 0, 2: 0}
        case = cases[int(matrix[i, j])]
        for right in case.keys():
            for left, count in case[right].items():
                newRightValues[left] += rightValues[right] * count
                
        rightValues = newRightValues
                    
sum(rightValues.values())

CPU times: total: 0 ns
Wall time: 0 ns


224

In [352]:
transitionRightTo1 = {'A': ['C', 'E'], 'B': ['C', 'E'], 'C': ['B'], 'D': ['C', 'E'],
                      'E': ['D'], 'F': ['B'], 'G': ['C', 'E'], 'H': ['D'],
                      'I': ['B'], 'K': ['D'], 'L': ['B'], 'N': ['D']} 

transitionRightTo0 = {'A': ['A', 'J'], 'B': ['A', 'J'], 'C': ['F', 'H', 'M'], 'D': ['A', 'J'],
                      'E': ['I', 'K', 'O'], 'F': ['F', 'H', 'M'], 'G': ['A', 'J'], 'H': ['I', 'K', 'O'],
                      'I': ['F', 'H', 'M'], 'J': ['G', 'L', 'N', 'P'], 'K': ['I', 'K', 'O'], 'L': ['F', 'H', 'M'], 
                      'M': ['G', 'L', 'N', 'P'], 'N': ['I', 'K', 'O'], 'O': ['G', 'L', 'N', 'P'], 'P': ['G', 'L', 'N', 'P']} 

transitionDownTo1 = {'A': ['D', 'E'], 'B': ['D', 'E'], 'C': ['D', 'E'], 'D': ['B'],
                     'E': ['C'], 'F': ['D', 'E'], 'G': ['B'], 'H': ['C'],
                     'J': ['C'], 'M': ['C'], 'I': ['B'], 'L': ['B']} 

transitionDownTo0 = {'A': ['A', 'K'], 'B': ['A', 'K'], 'C': ['A', 'K'], 'D': ['G', 'H', 'N'],
                      'E': ['I', 'J', 'O'], 'F': ['A', 'K'], 'G': ['G', 'H', 'N'], 'H': ['I', 'J', 'O'],
                      'I': ['G', 'H', 'N'], 'J': ['I', 'J', 'O'], 'K': ['F', 'L', 'M', 'P'], 'L': ['G', 'H', 'N'], 
                      'M': ['I', 'J', 'O'], 'N': ['F', 'L', 'M', 'P'], 'O': ['F', 'L', 'M', 'P'], 'P': ['F', 'L', 'M', 'P']} 

transitionLeftTo1 = {'A': ['B', 'D'],'C': ['B', 'D'], 'E': ['B', 'D'], 'J': ['B', 'D'],
                     'B': ['C'], 'F': ['C'], 'H': ['C'], 'M': ['C'],
                     'D': ['E'], 'I': ['E'], 'K': ['E'], 'O': ['E']} 

transitionLeftTo0 = {'A': ['A', 'G'], 'C': ['A', 'G'], 'E': ['A', 'G'], 'J': ['A', 'G'],
                     'B': ['F', 'I', 'L'], 'F': ['F', 'I', 'L'], 'H': ['F', 'I', 'L'], 'M': ['F', 'I', 'L'],
                     'D': ['H', 'K', 'N'], 'I': ['H', 'K', 'N'], 'K': ['H', 'K', 'N'], 'O': ['H', 'K', 'N'],
                     'G': ['J', 'O', 'M', 'P'], 'L': ['J', 'O', 'M', 'P'], 'N': ['J', 'O', 'M', 'P'],
                     'P': ['J', 'O', 'M', 'P']} 

transitionUpTo1 = {'A': ['B', 'C'], 'D': ['B', 'C'], 'K': ['B', 'C'],
                   'B': ['D'], 'G': ['D'], 'H': ['D'], 'N': ['D'],
                   'C': ['E'], 'I': ['E'], 'J': ['E'], 'O': ['E']} 

transitionDownTo0 = {'A': ['A', 'K'], 'B': ['A', 'K'], 'C': ['A', 'K'], 'D': ['G', 'H', 'N'],
                      'E': ['I', 'J', 'O'], 'F': ['A', 'K'], 'G': ['G', 'H', 'N'], 'H': ['I', 'J', 'O'],
                      'I': ['G', 'H', 'N'], 'J': ['I', 'J', 'O'], 'K': ['F', 'L', 'M', 'P'], 'L': ['G', 'H', 'N'], 
                      'M': ['I', 'J', 'O'], 'N': ['F', 'L', 'M', 'P'], 'O': ['F', 'L', 'M', 'P'], 'P': ['F', 'L', 'M', 'P']} 

transitionsRight = [transitionRightTo0, transitionRightTo1]
transitionsDown = [transitionDownTo0, transitionDownTo1]
transitionsLeft = [transitionLeftTo0, transitionLeftTo1]

In [359]:
def merge(states_X, states_Y, transitions):
    states_Z = defaultdict(lambda: 0) # res
    for state_xi, count in states_X.items():
        if state_xi in transitions.keys():
            for state_next in transitions[state_xi]:
                if state_next in states_Y.keys():
                    states_Z[state_next] += count * states_Y[state_next]
                    
    return states_Z

def mergeInplace(states_X, states_Y):
    states_Z = defaultdict(lambda: 0) # res
    for state_xi, count in states_X.items():
        if state_xi in states_Y.keys():
            states_Z[state_xi] += count * states_Y[state_xi]
                    
    return states_Z

def mergeMin(states_X, states_Y):
    states_Z = defaultdict(lambda: 0) # res
    for state_xi, count in states_X.items():
        if state_xi in states_Y.keys():
            states_Z[state_xi] += min(count, states_Y[state_xi])
                    
    return states_Z

def mergeConstraint(states_X, states_Y, transitions, transitionsUp):
    states_Z = defaultdict(lambda: 0) # res
    for state_xi, count in states_X.items():
        if state_xi in transitions.keys():
            for state_next in transitions[state_xi]:
                if state_next in states_Y.keys() and state_next in transitionsUp.keys():
                    states_Z[state_next] += count
                    
    return states_Z

In [306]:
matrix = [[1], [0], [1]]

initial = [{'A': 1, 'F': 1, 'G': 1, 'H': 1, 'I': 1, 'J': 1, 'K': 1, 'L': 1, 'M': 1, 'N': 1, 'O': 1, 'P': 1},
           {'B': 1, 'C': 1, 'D': 1, 'E': 1}]

curStates = initial[int(matrix[0][0])]

for row in range(1, len(matrix)):
    case = int(matrix[row][0])
    transitions = transitionsDown[case]
    curStates = merge(curStates, initial[case], transitions)

print(sum(curStates.values()))

8


In [366]:
#%%time

matrix = [[1, 0],
          [0, 1]]

initial = [{'A': 1, 'F': 1, 'G': 1, 'H': 1, 'I': 1, 'J': 1, 'K': 1, 'L': 1, 'M': 1, 'N': 1, 'O': 1, 'P': 1},
           {'B': 1, 'C': 1, 'D': 1, 'E': 1}]

curStates = initial[int(matrix[0][0])]

# for row in range(len(matrix[0])):
#     for col in range(1, len(matrix[0])):
#         case = int(matrix[row][col])
#         transitions = transitionsRight[case]
#         curStates = merge(curStates, initial[case], transitions)

left2right = [initial[int(matrix[0][0])]]
right2left = [initial[int(matrix[0][-1])]]
both = []
for col in range(1, len(matrix[0])):
    case1 = int(matrix[0][col])
    transitions1 = transitionsRight[case1]
    
    case2 = int(matrix[0][-col - 1])
    transitions2 = transitionsLeft[case2]
    
    left2right.append(merge(left2right[col - 1], initial[case1], transitions1))
    right2left.append(merge(right2left[col - 1], initial[case2], transitions2))

for col in range(len(matrix[0])):
    both.append(mergeInplace(left2right[col], right2left[-col - 1]))
    
#right2left
print(sum(right2left[-1].values()))
print(sum(left2right[-1].values()))
print(right2left[0])
print(left2right[-1])

12
10
{'A': 1, 'F': 1, 'G': 1, 'H': 1, 'I': 1, 'J': 1, 'K': 1, 'L': 1, 'M': 1, 'N': 1, 'O': 1, 'P': 1}
defaultdict(<function merge.<locals>.<lambda> at 0x0000022400872050>, {'A': 2, 'J': 2, 'F': 1, 'H': 1, 'M': 1, 'I': 1, 'K': 1, 'O': 1})


In [367]:
for elem in both:
    print(sum(elem.values()))

10
10


In [361]:
nextLine = []
for col in range(len(both)):
    case = int(matrix[1][col])
    transitions = transitionsDown[case]
    nextLine.append(merge(both[col], initial[case], transitions))

In [362]:
z = [mergeMin(nextLine[1], initial[0], transitionsLeft[0]), merge(nextLine[0], initial[1], transitionsRight[1])]
for elem in z:
    print(sum(elem.values()))
    

26
26


In [364]:
sum(merge(nextLine[1], initial[0], transitionsLeft[0]).values())

15

In [336]:
for elem in nextLine:
    print(elem)
    print(sum(elem.values()))

defaultdict(<function merge.<locals>.<lambda> at 0x0000022400873010>, {'A': 5, 'K': 5, 'G': 2, 'H': 2, 'N': 2, 'I': 3, 'J': 3, 'O': 3})
25
defaultdict(<function merge.<locals>.<lambda> at 0x0000022400872B00>, {'D': 3, 'E': 3, 'C': 4, 'B': 1})
11


In [ ]:
A = [[0, 0],  
     [0, 0]]

# 1

B = [[1, 0],
     [0, 0]]

C = [[0, 1],
     [0, 0]]

D = [[0, 0],
     [1, 0]]

E = [[0, 0],
     [0, 1]]

# 2

F = [[1, 1],
     [0, 0]]

G = [[1, 0],
     [1, 0]]

H = [[1, 0],
     [0, 1]]

I = [[0, 1],
     [1, 0]]

J = [[0, 1],
     [0, 1]]

K = [[0, 0],
     [1, 1]]


# 3

L = [[1, 1],
     [1, 0]]

M = [[1, 1],
     [0, 1]]

N = [[1, 0],
     [1, 1]]

O = [[0, 1],
     [1, 1]]


# 4

P = [[1, 1],
     [1, 1]]


In [368]:
import itertools
allPreimagesCol=[]
matches=[]
count=0
totalpaths=[]
found_matches={}

match_side=dict({0:[0,1,4,5],
                1:[2,3,6,7],
                2:[0,1,4,5],
                3:[2,3,6,7],
                4:[8,9,12,13],
                5:[10,11,14,15],
                6:[8,9,12,13],
                7:[10,11,14,15],
                8:[0,1,4,5],
                9:[2,3,6,7],
                10:[0,1,4,5],
                11:[2,3,6,7],
                12:[8,9,12,13],
                13:[10,11,14,15],
                14:[8,9,12,13],
                15:[10,11,14,15]})

match_down=dict({0:[0,1,2,3],
                 1:[4,5,6,7],
                 2:[8,9,10,11],
                 3:[12,13,14,15],
                 4:[0,1,2,3],
                 5:[4,5,6,7],
                 6:[8,9,10,11],
                 7:[12,13,14,15],
                 8:[0,1,2,3],
                 9:[4,5,6,7],
                 10:[8,9,10,11],
                 11:[12,13,14,15],
                 12:[0,1,2,3],
                 13:[4,5,6,7],
                 14:[8,9,10,11],
                 15:[12,13,14,15]
                 })

paths={}
sides={}

rules=dict({0:[0,3,5,6,7,9,10,11,12,13,14,15],
             1:[1,2,4,8]})
#for every i,j the rulebase follows the order [(i,j),(i,j+1),(i+1,j),(i+1,j+1),nextstate]
rulebase=[[0,0,0,0,False],
          [0,0,0,1,True],
          [0,0,1,0,True],
          [0,0,1,1,False],
          [0,1,0,0,True],
          [0,1,0,1,False],
          [0,1,1,0,False],
          [0,1,1,1,False],
          [1,0,0,0,True],
          [1,0,0,1,False],
          [1,0,1,0,False],
          [1,0,1,1,False],
          [1,1,0,0,False],
          [1,1,0,1,False],
          [1,1,1,0,False],
          [1,1,1,1,False]]

def getAllPathsCol(col, u, depth, path): 
    global matches
    depth+=1
    path.append(u) 
    if depth==length:
        matches.append(path[:])
    else: 
        for i in match_down[u]:
            if(rulebase[i][4]==col[depth]):
                    getAllPathsCol(col,i, depth, path)
    path.pop()

def match(row,depth):
    global count
    global length
    global newg
    prods=[]
    nextl=[]
    depth+=1
    if(depth<length):
        vaar=tuple((tuple(row)+tuple(tuple(i) for i in allPreimagesCol[depth] )))
    if(depth==length):
        count+=1
    elif( vaar in found_matches):
        for p in found_matches[vaar]:
            match(list(p),depth)
    else:
        if(tuple(row) in sides):
            prods=sides[tuple(row)]
        else:
            prods=[l for l in itertools.product(*[match_side[k] for k in row])]
        m=[]
        for i in allPreimagesCol[depth]:
            m.append((tuple(i)))
        nextl=[q for q in set(m)]
        s=set(prods) & set(nextl)
        if(len(s)>0):
            found_matches[vaar]=s
            for j in (list(s)):
                    match(list(j),depth)


def findMatch(row):
    for i in row:
        if(tuple((tuple(i)+tuple([tuple(j) for j in allPreimagesCol[0] ]))) in found_matches):
            for p in found_matches[tuple((tuple(i)+tuple([tuple(j) for j in allPreimagesCol[0] ])))]:
                match(list(p),0)
        else:
            match(i,0)



def solution(g):
    # Okay I had to search "expanding nebula foobar", but I found the concept to be "Cellular Automaton" - 
    # Nature of Code by Daniel Shiffman is a great resource.
    # The question wants it in reverse. 
    col=[] 
    global length
    newg=list(zip(*g))
    global allPreimagesCol
    global matches

    for i in range(len(newg)):
        col=newg[i]
        length=len(col)
        if(tuple(col) in paths):
            allPreimagesCol.append(paths[tuple(col)])
        else:
            for item in rules[col[0]]:
                path=[]
                getAllPathsCol(col,item,0,path)
            allPreimagesCol.append(matches)
            paths[tuple(col)]=matches
            matches=[]
    row=allPreimagesCol[0]
    length=len(allPreimagesCol)
    findMatch(row)
    return(count)

In [374]:
m = [[True, False, True],
     [False, True, False],
     [True, False, True]]

m = [[True, True, False, True, False, True, False, True, True, False],
     [True, True, False, False, False, False, True, True, True, False],
     [True, True, False, False, False, False, False, False, False, True],
     [False, True, False, False, False, False, True, True, False, False]]

# m = [[True, False, True, False, False, True, True, True],
#      [True, False, True, False, False, False, True, False],
#      [True, True, True, False, False, False, True, False],
#      [True, False, True, False, False, False, True, False],
#      [True, False, True, False, False, True, True, True]]

solution(m)

12

In [370]:
m = [[True, False, True],
     [False, True, False],
     [True, False, True]]